# Polymorphisme
(se base sur https://github.com/ninjaaron/oo-and-polymorphism-in-julia/)

### Héritage avec des types abstraits

Crééons un type abstrait qui part du principe que les types héritant de shape doivent proposer une méthode area. Cependant, on ne peut pas forcer le sous-types à avoir la méthode area, il faut donc bien documenter le code

In [5]:
abstract type Shape end
combined_area(a::Shape,b::Shape) = area(a) + area(b)

combined_area (generic function with 1 method)

Structure héritant de Shape :

In [3]:
struct Circle <: Shape
    diameter::Float64
end
radius(c::Circle) = c.diameter/2
area(c::Circle) = pi*radius(c)^2

area (generic function with 1 method)

Ajoutons un type qui hérite du type Shape

In [4]:
abstract type AbstractRectangle <: Shape end
area(r::AbstractRectangle) = width(r) * height(r)

area (generic function with 2 methods)

Structure Rectangle héritant de AbstractRectangle :

In [12]:
struct Rectangle <: AbstractRectangle
    w::Float64
    h::Float64
end

width(r::Rectangle) = r.w
height(r::Rectangle) = r.h

height (generic function with 2 methods)

Autre structure héritant d'AbstractRectangle

In [8]:
struct Square <: AbstractRectangle
    length::Float64
end
width(s::Square) = s.length
height(s::Square) = s.length

height (generic function with 1 method)

Tests :

In [14]:
c = Circle(3)
s = Square(2)
r = Rectangle(3,5)

@show combined_area(c,s)
@show combined_area(s,r)

combined_area(c, s) = 11.068583470577035
combined_area(s, r) = 19.0


19.0

### Composition et transfert de méthodes : une alternative à l'héritage.

La stratégie pour l'héritage de composition (compositiona linheritance) est de créer des structures qui ont le type de l'interface requise en tant que champ, plutôt qu'en tant que super type, et ensuite on peut quand nécessaire, déléguer certaines méthodes en les forwardant automatiquement. Julia rend ça plus simple avec le mot clé eval, car on peut forward les méthodes avec un boucle. C'est typique de Julia d'utiliser eval pour des prétraitement de ce type.

In [15]:
struct HasInterestingField
    data::String
end

double(hif::HasInterestingField) = hif.data^2
shout(hif::HasInterestingField) = uppercase(string(hif.data,"!"))

shout (generic function with 1 method)

In [17]:
# Méthode de composition pour récupérer ce qui nous intéresse :
struct WantsInterestingField
    interesting::HasInterestingField
    # Changement du constructeur pour directement prendre un string data en entrée et non un HasInterestingField
    WantsInterestingField(data) = new(HasInterestingField(data))
end

# forward method
for method in (:double, :shout)
    @eval $method(wif::WantsInterestingField) = $method(wif.interesting)
end

Pour le forwarding, on utilise un peu de metaprogramming : 
@eval est une macro, un mécanisme qui va générer du code à la compilation.
:double est en fait une expression qui désigne la méthode double lorsqu'elle est évaluée
$method permet de prendre la valeur de la variable method (ici ce sera :double ou :shout) (c'est ce qui est appelé de l'interpolation)



Tests :

In [18]:
wif = WantsInterestingField("coucou")
@show shout(wif)
@show double(wif)

shout(wif) = "COUCOU!"
double(wif) = "coucoucoucou"


"coucoucoucou"

  ### Generics: Statically typed dynamic typing

En Julia, les types génériques peuvent être utilisé par sécurité, mais ils sont aussi la clé pour faire des structures qui sont efficientes et polymorphiques

In [20]:
struct GenPoint{T}
    x::T
    y::T
end
# Ça force x et y à être du même type T

GenPoint(2,3)

GenPoint{Int64}(2, 3)

In [21]:
# On peut aussi faire avec plusieurs types :
struct GenPoint2{X,Y}
    x::X
    y::Y
end
GenPoint2(3,2.0)


GenPoint2{Int64, Float64}(3, 2.0)

On peut tout de même contraindre le type de T :

In [22]:
struct Point{T<:Real}
    x::T
    y::T
end

@show Point(1,4)
@show Point(4.0,2.4)

Point(1, 4) = Point{Int64}(1, 4)
Point(4.0, 2.4) = Point{Float64}(4.0, 2.4)


Point{Float64}(4.0, 2.4)

Il est préférable de faire Point{T<:Real} puis avec des attributs de type T que faire une classe non générique avec des attributs de type Real car Real n'est pas un type concret mais abstrait, le compilateur Julia ne peut donc pas faire d'assomption dessus, la sélection de méthode se fera alors à l'exécution plutôt qu'à la compilation.

Pour de bonnes performances, utiliser les génériques comme dans l'exemple précédent avec des champs de type T est nécessaire, contrairement aux signatures des méthodes. Normalement, si la stabilité des types est respectée, le compilateur sait le type de n'importe quoi et ne repose pas sur les types de la méthode pour optimiser les performances.

Exemple d'implémentation d'une linked list en Julia qui utilise les types génériques :

In [26]:
struct Nil end #nil = nihil en latin = nothing -> pour signifier la fin de la liste

struct List{T}
    head::T
    tail::Union{List{T},Nil} # tail est soit un autre noeud (List{N}) soit Nil (fin de la liste)
    # Union permet d'accepter les 2 types
end

# création d'une liste depuis un tableau 
mklist(array::AbstractArray{T}) where T = foldr(List{T},array, init=Nil())
# on déclare diretctement le type T dans la signature de la fonction
# it turns whatever the element type of the array is into a value in the function body.
# foldr permet de traiter la collection array en créant des List{T} successivement en commençant par Nil


# implémentation du protocol d'itération https://docs.julialang.org/en/v1/base/collections/#lib-collections-iteration-1
Base.iterate(l::List) = iterate(l,l)
Base.iterate(::List,l::List) = l.head, l.tail #pas de nom au premier argument car sa valeur n'est pas utilisée
Base.iterate(::List,::Nil) = nothing

Test :

In [25]:
l1 = mklist(1:3)
l2 = mklist(["a",2])

@show l1
@show l2

for val in l1
    println(val)
end

l1 = List{Int64}(1, List{Int64}(2, List{Int64}(3, Nil())))
l2 = List{Any}("a", List{Any}(2, Nil()))
1
2
3


### Le pattern Trait

Les traits sont en quelque sorte un contrat avec le compilateur selon lequel un certain type va implémenter les bonnes interfaces, et peut donc être utilisé comme argument pour toute fonction qui requiert cette interface. C'est une alternative à l'héritage classique.

Dans sa forme la plus simple, un trait est juste une structure vide et une dispatch qui lance une erreur dans le cas le plus général. Vous vérifiez ensuite l'existence du trait en utilisant le constructeur dans une méthode dispatch.

In [27]:
struct Zlurmable end
Zlurmable(::T) where T = error("Type $T doesn't implement the Zlurmable trait")

zlurm(x) = zlurm(Zlurmable(x),x)
zlurm(::Zlurmable,x) = x+1

zlurm (generic function with 2 methods)

Pour l'instant, il n'y a pas de types qui implémentent le Zlurmable trait, on ne peut donc pas utiliser la fonction zlurm (zlurm(1) donne une erreur car Int64 n'implément pas ZLurmable par exemple)

In [28]:
# What we need to do is add the trait to a type:
Zlurmable(::Int64) = Zlurmable()
# et maintenant :
zlurm(3)


4